# Part 10: Working with Timeseries Data

In [2]:
import polars as pl
import numpy as np
pl.Config.set_tbl_rows(10)
pl.Config.set_fmt_str_lengths(50)
from datetime import datetime

url = 'https://raw.githubusercontent.com/CoreyMSchafer/code_snippets/master/Python/Pandas/10-Datetime-Timeseries/ETH_1h.csv'

df = pl.read_csv(url)
df

Date,Symbol,Open,High,Low,Close,Volume
str,str,f64,f64,f64,f64,f64
"""2020-03-13 08-PM""","""ETHUSD""",129.94,131.82,126.87,128.71,1.9407e6
"""2020-03-13 07-PM""","""ETHUSD""",119.51,132.02,117.1,129.94,7.5797e6
"""2020-03-13 06-PM""","""ETHUSD""",124.47,124.85,115.5,119.51,4.8987e6
"""2020-03-13 05-PM""","""ETHUSD""",124.08,127.42,121.63,124.47,2.7535e6
"""2020-03-13 04-PM""","""ETHUSD""",124.85,129.51,120.17,124.08,4.4614e6
…,…,…,…,…,…,…
"""2017-07-01 03-PM""","""ETHUSD""",265.74,272.74,265.0,272.57,1.5003e6
"""2017-07-01 02-PM""","""ETHUSD""",268.79,269.9,265.0,265.74,1.7025e6
"""2017-07-01 01-PM""","""ETHUSD""",274.83,274.93,265.0,268.79,3.0108e6


In [3]:
# Target first row to see that it's date
df[0,'Date']

'2020-03-13 08-PM'

In [4]:
df.select(pl.col('Date'))[0]

Date
str
"""2020-03-13 08-PM"""


In [5]:
# Get day name
df.select(pl.col('Date').dt.weekday())[0]

InvalidOperationError: `weekday` operation not supported for dtype `str`

In [6]:
# skip this!
from datetime import datetime

date_string = '2020-03-13 08-PM'
parsed_date = datetime.strptime(date_string, '%Y-%m-%d %I-%p')
print(parsed_date)

2020-03-13 20:00:00


In [7]:
# Converting string to date. This worked!

def convert_date(df):
    df = df.with_columns(pl.col('Date').map_elements(lambda date_str: datetime.strptime(date_str, '%Y-%m-%d %I-%p')))
    return df

df = df.pipe(convert_date)
df

C:\Users\Jmutenge\AppData\Local\Temp\ipykernel_22208\1694169782.py:4: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - pl.col("Date").map_elements(lambda date_str: ...)
  + pl.col("Date").str.to_datetime(format="%Y-%m-%d %I-%p")

  df = df.with_columns(pl.col('Date').map_elements(lambda date_str: datetime.strptime(date_str, '%Y-%m-%d %I-%p')))


Date,Symbol,Open,High,Low,Close,Volume
datetime[μs],str,f64,f64,f64,f64,f64
2020-03-13 20:00:00,"""ETHUSD""",129.94,131.82,126.87,128.71,1.9407e6
2020-03-13 19:00:00,"""ETHUSD""",119.51,132.02,117.1,129.94,7.5797e6
2020-03-13 18:00:00,"""ETHUSD""",124.47,124.85,115.5,119.51,4.8987e6
2020-03-13 17:00:00,"""ETHUSD""",124.08,127.42,121.63,124.47,2.7535e6
2020-03-13 16:00:00,"""ETHUSD""",124.85,129.51,120.17,124.08,4.4614e6
…,…,…,…,…,…,…
2017-07-01 15:00:00,"""ETHUSD""",265.74,272.74,265.0,272.57,1.5003e6
2017-07-01 14:00:00,"""ETHUSD""",268.79,269.9,265.0,265.74,1.7025e6
2017-07-01 13:00:00,"""ETHUSD""",274.83,274.93,265.0,268.79,3.0108e6


In [8]:
# Get day name
df.select(pl.col('Date').dt.weekday())[0]

Date
u32
5


In [10]:
# Create a column based that will have day of week

df = df.with_columns(pl.col('Date').dt.weekday().alias('DayOfWeek'))
df
 

Date,Symbol,Open,High,Low,Close,Volume,DayOfWeek
datetime[μs],str,f64,f64,f64,f64,f64,u32
2020-03-13 20:00:00,"""ETHUSD""",129.94,131.82,126.87,128.71,1.9407e6,5
2020-03-13 19:00:00,"""ETHUSD""",119.51,132.02,117.1,129.94,7.5797e6,5
2020-03-13 18:00:00,"""ETHUSD""",124.47,124.85,115.5,119.51,4.8987e6,5
2020-03-13 17:00:00,"""ETHUSD""",124.08,127.42,121.63,124.47,2.7535e6,5
2020-03-13 16:00:00,"""ETHUSD""",124.85,129.51,120.17,124.08,4.4614e6,5
…,…,…,…,…,…,…,…
2017-07-01 15:00:00,"""ETHUSD""",265.74,272.74,265.0,272.57,1.5003e6,6
2017-07-01 14:00:00,"""ETHUSD""",268.79,269.9,265.0,265.74,1.7025e6,6
2017-07-01 13:00:00,"""ETHUSD""",274.83,274.93,265.0,268.79,3.0108e6,6


In [11]:
# Get earliest date
df['Date'].min()

datetime.datetime(2017, 7, 1, 11, 0)

In [12]:
df['Date'].max()

datetime.datetime(2020, 3, 13, 20, 0)

In [13]:
# What is the time delta
df['Date'].max() - df['Date'].min()

datetime.timedelta(days=986, seconds=32400)

In [18]:
# show for certain range dates (from Jan 2020 and later)
df.filter(pl.col('Date') > datetime(2020,1,1))

Date,Symbol,Open,High,Low,Close,Volume,DayOfWeek
datetime[μs],str,f64,f64,f64,f64,f64,u32
2020-03-13 20:00:00,"""ETHUSD""",129.94,131.82,126.87,128.71,1.9407e6,5
2020-03-13 19:00:00,"""ETHUSD""",119.51,132.02,117.1,129.94,7.5797e6,5
2020-03-13 18:00:00,"""ETHUSD""",124.47,124.85,115.5,119.51,4.8987e6,5
2020-03-13 17:00:00,"""ETHUSD""",124.08,127.42,121.63,124.47,2.7535e6,5
2020-03-13 16:00:00,"""ETHUSD""",124.85,129.51,120.17,124.08,4.4614e6,5
…,…,…,…,…,…,…,…
2020-01-01 05:00:00,"""ETHUSD""",129.56,129.94,129.47,129.83,232163.7,3
2020-01-01 04:00:00,"""ETHUSD""",129.57,130.0,129.5,129.56,702786.82,3
2020-01-01 03:00:00,"""ETHUSD""",130.37,130.44,129.38,129.57,496704.23,3


In [31]:
# CAN'T DO AND (&) OPERATIONS WITH DATES
# Show between 2019 all the way to the end of the month of Jan in 2020

(df
 .filter(pl.col('Date') >= datetime(2019,1,1))
 .filter(pl.col('Date') < datetime(2020,1,31))
 )

Date,Symbol,Open,High,Low,Close,Volume,DayOfWeek
datetime[μs],str,f64,f64,f64,f64,f64,u32
2020-01-30 23:00:00,"""ETHUSD""",186.62,186.89,182.99,184.55,1.262371e6,4
2020-01-30 22:00:00,"""ETHUSD""",185.03,186.63,183.9,186.62,992325.34,4
2020-01-30 21:00:00,"""ETHUSD""",184.4,185.03,183.19,185.03,701167.77,4
2020-01-30 20:00:00,"""ETHUSD""",181.26,185.14,181.26,184.4,2.1802e6,4
2020-01-30 19:00:00,"""ETHUSD""",180.91,181.99,180.2,181.26,1.3786e6,4
…,…,…,…,…,…,…,…
2019-01-01 04:00:00,"""ETHUSD""",130.75,133.96,130.74,131.96,2.7911e6,2
2019-01-01 03:00:00,"""ETHUSD""",130.06,130.79,130.06,130.75,503732.63,2
2019-01-01 02:00:00,"""ETHUSD""",130.79,130.88,129.55,130.06,838183.43,2


In [37]:
# Avg close for a specific date range
(df
 .filter(pl.col('Date') >= datetime(2020,1,1))
 .filter(pl.col('Date') < datetime(2020,2,29))
 .select('Close')
 .mean()
 )

Close
f64
194.63988


In [51]:
# GET MORE GRANULAR
# See the Highs for Jan 1 2020 (New Year's)
# Should have 24 records

(df
 .filter(pl.col('Date') >= datetime(2020,1,1,00,00,00))
 .filter(pl.col('Date') < datetime(2020,1,31,23,59,00))
 .select('Close')
 .mean()
 )

Close
f64
155.085739


In [84]:
(df
 .filter(pl.col('Date') >= datetime(2020,1,1,00,00,00))
 .filter(pl.col('Date') < datetime(2020,1,31,23,59,00))
 .sort('Date', descending=False) #requires dates to be sorted.
#  .upsample(time_column="Date", every="30m", maintain_order=True) #creates nulls because our data is hourly.
 .group_by_dynamic('Date', every='1d') # Max for each day
 .agg(pl.col('High').max())
 )

Date,High
datetime[μs],f64
2020-01-01 00:00:00,132.68
2020-01-02 00:00:00,130.19
2020-01-03 00:00:00,134.93
2020-01-04 00:00:00,135.75
2020-01-05 00:00:00,138.97
…,…
2020-01-27 00:00:00,171.7
2020-01-28 00:00:00,176.5
2020-01-29 00:00:00,178.31


In [102]:
# Show weekly mean for all columns
(df
 .sort('Date', descending=False)
 .group_by_dynamic('Date', every='1d')
 .agg(pl.col(['Open','High','Low','Close','Volume']).mean())
 )

Date,Open,High,Low,Close,Volume
datetime[μs],f64,f64,f64,f64,f64
2017-07-01 00:00:00,266.581538,268.698462,261.861538,265.284615,2.1988e6
2017-07-02 00:00:00,268.870833,272.43875,266.422083,269.7825,2.1776e6
2017-07-03 00:00:00,279.241667,280.774167,276.599583,278.882083,1.4590e6
2017-07-04 00:00:00,277.042083,278.034167,275.1175,276.743333,1.2546e6
2017-07-05 00:00:00,265.147917,267.12,262.834583,265.025833,1.7979e6
…,…,…,…,…,…
2020-03-09 00:00:00,200.680417,203.311667,197.86,200.818333,3.8048e6
2020-03-10 00:00:00,201.672083,203.155,199.9125,201.5775,1.5531e6
2020-03-11 00:00:00,196.155833,197.684167,194.100833,195.910417,1.8949e6


In [107]:
# Get max and sum
pl.Config.set_fmt_float('full')

(df
 .sort('Date', descending=False)
 .group_by_dynamic('Date', every='1d')
 .agg([pl.col('Open').max(),
       pl.col('Volume').sum()])
 )

Date,Open,Volume
datetime[μs],f64,f64
2017-07-01 00:00:00,279.98,28584024.81
2017-07-02 00:00:00,292.54,52262287.779999994
2017-07-03 00:00:00,285,35016353.53000001
2017-07-04 00:00:00,282.58,30109905.280000005
2017-07-05 00:00:00,271,43150162.52
…,…,…
2020-03-09 00:00:00,207.79,91314036.65000002
2020-03-10 00:00:00,205.79,37273515.54
2020-03-11 00:00:00,202.48,45476825.550000004
